**SECTION 1 : IMPORT LIBRARY**

In [2]:
from IPython import get_ipython
from IPython.display import display

In [3]:
import requests  #This libraray helps us to fetch data from API
import pandas as pd #used for handling and analysing data
import numpy as np #for numerical opertaions
from sklearn.model_selection import train_test_split #to split data into tarining and testing data
from sklearn.preprocessing import LabelEncoder #to convert categorical data into numericals values
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor #models for classification and regression tasks
from sklearn.metrics import mean_squared_error #to measure the accuracy of prediction
from datetime import datetime, timedelta # to handle date and time
import pytz

In [4]:
API_KEY = '5dea6d558c8b6b29eaaf31297abb95d0' #replace with our actual API key
BASE_URL ='https://api.openweathermap.org/data/2.5/' #base url for making API requests

**1. Fetch current Weather Data**

In [5]:
def get_current_weather(city):
  url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric" # construct the API request URL
  response =requests.get(url)  #send the get request to API
  data = response.json()
  return{
      'city': data['name'],
      'current_temp' : round(data['main']['temp']),
      'feels_like' : round(data['main']['feels_like']),
      'temp_min' : round(data['main']['temp_min']),
      'temp_max' : round(data['main']['temp_max']),
      'humidity' : round(data['main']['humidity']),
      'description' : data['weather'][0]['description'],
     # 'country' : data['sys'][0]['country'],
      'country' :[ 'country'],
      'wind_gust_dir':data['wind']['deg'],
      'pressure': data['main']['pressure'],
      'Wind_Gust_Speed': data['wind']['speed'],

  }

**2. Read Historical Data**

In [6]:
def read_historical_data(filename):
  df= pd.read_csv(filename)
  df =df.dropna()
  df =df.drop_duplicates()
  return df

**3. Prepare Data for Training**

In [7]:
def prepare_data(data):
  le = LabelEncoder() #create a labelencoder instance
  data['WindGustDir']= le.fit_transform(data['WindGustDir'])
  data['RainTomorrow']= le.fit_transform(data['RainTomorrow'])

  #define the feature variable and target variable
  x= data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed','Humidity', 'Pressure','Temp']]
  y=data['RainTomorrow']
  return x, y, le #return feature variable, target variable and the label  encoder


**4. Train Rain Prediction Model**

In [8]:
def train_rain_model(x,y):
  x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)
  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(x_train,y_train) #train the model

  y_pred =model.predict(x_test) #to make prediction on test set

  print("Mean Squared Error for Rain Model")

  print(mean_squared_error(y_test,y_pred))

  return model


**5. Preapare regression data**

In [9]:
def prepare_regression_data(data, feature):
  x,y =[],[] #intialize list for feature and target value

  for i in range(len(data)-1):
    x.append(data[feature].iloc[i])
    y.append(data[feature].iloc[i+1])

  x=np.array(x).reshape(-1, 1)
  y=np.array(y)
  return x, y

**Train Regression Data**

In [10]:
def train_regression_model(x,y):
  model=RandomForestRegressor(n_estimators=100,random_state=42)
  model.fit(x,y)

  return model

**Predict Future**

In [11]:
def predict_feature(model,current_value):
  predictions=[current_value]

  for i in range(5):
    next_value=model.predict(np.array([[predictions[-1]]]))

    predictions.append(next_value[0])
  return predictions[1:]

**WEATHER ANALYSIS FUNCTIONS**

In [12]:
def weather_view():
  city = input("Enter the city name: ")
  current_weather = get_current_weather(city)

  #load historical data
  historical_data = read_historical_data('/content/drive/MyDrive/weather.csv')

  #prepare and train prediction model
  x,y,le=prepare_data(historical_data)

  rain_model= train_rain_model(x, y)

  #map wind direction to campass points
  wind_deg =current_weather['wind_gust_dir']%360
  compass_points= [
      ("N",0, 11.25),("NNE", 11.25, 33.75),("NE", 33.75, 56.25),("ENE", 56.25, 78.75),("E",78.75, 101.25),
      ("ESE",101.25, 123.75),("SE", 123.75, 146.25),("SSE",146.25, 168.75),("S",168.75, 191.25),
      ("SSW",191.25, 213.75),("SW",213.75, 236.25),("WSW",236.25, 258.75),("W", 258.75, 281.25),
      ("WNW",281.25, 303.75),("NW",303.75, 326.25),("NNW",326.25, 348.75)

  ]
  compass_direction= next(point for point, start,end in compass_points if start <= wind_deg < end)

  compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

  current_data = {
      'MinTemp': current_weather['temp_min'],
      'MaxTemp': current_weather['temp_max'],
      'WindGustDir': compass_direction_encoded,
      'WindGustSpeed': current_weather['Wind_Gust_Speed'],
      'Humidity': current_weather['humidity'],
      'Pressure': current_weather['pressure'],
      'Temp': current_weather['current_temp']
  }

  current_df = pd.DataFrame([current_data])

  #make rain prediction
  rain_prediction = rain_model.predict(current_df)[0]

  #prepare regression model for tempersture and humidity
  x_temp,y_temp = prepare_regression_data(historical_data, 'Temp')
  x_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')
  temp_model = train_regression_model(x_temp, y_temp)
  hum_model = train_regression_model(x_hum, y_hum)

  #predict future temperature and humidity
  future_temp = predict_feature(temp_model, current_weather['temp_min'])
  future_humidity = predict_feature(hum_model, current_weather['humidity'])

  #prepare time for furture prediction
  timezone = pytz.timezone('Asia/Karachi')
  now =datetime.now(timezone)
  next_hour = now+timedelta(hours=1)
  next_hour = next_hour.replace(minute=0,second=0,microsecond=0)

  future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

  #display results
  print(f"City: {city}, {current_weather['country']}")
  print(f"Current Temperature: {current_weather['current_temp']}°C")
  print(f"Feels Like: {current_weather['feels_like']}°C")
  print(f"Minimum Temperature: {current_weather['temp_min']}°C")
  print(f"Maximum Temperature : {current_weather['temp_max']}°C")
  print(f"Humidity: {current_weather['humidity']}%")
  print(f"Weather Description: {current_weather['description']}")
  print(f"Rain Presiction : {'Yes' if rain_prediction else 'No'}")

  print("\nFuture Temperature Predictions:")
  for time, temp in zip(future_times, future_temp):
    print(f"{time}: {round(temp, 1)}°C")

  print("\nFuture Humidity Predictions:")
  for time, humidity in zip(future_times, future_humidity):
    print(f"{time}: {round(humidity, 1)}%")

weather_view()







Enter the city name: india
Mean Squared Error for Rain Model
0.1506849315068493
City: india, ['country']
Current Temperature: 7°C
Feels Like: 6°C
Minimum Temperature: 7°C
Maximum Temperature : 7°C
Humidity: 85%
Weather Description: clear sky
Rain Presiction : Yes

Future Temperature Predictions:
11:00: 8.5°C
12:00: 9.5°C
13:00: 11.6°C
14:00: 11.7°C
15:00: 11.6°C

Future Humidity Predictions:
11:00: 49.0%
12:00: 48.6%
13:00: 48.6%
14:00: 48.6%
15:00: 48.6%
